In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
import torchvision
import torchvision.transforms as transforms


In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import os
import cv2
from tqdm import tqdm
from sklearn.utils import shuffle

In [3]:
train_dir = '/kaggle/input/seen-dataset-12-class/Seen Datasets/train'
test_dir = '/kaggle/input/seen-dataset-12-class/Seen Datasets/val'
# pred_dir = '../input/intel-image-classification/seg_pred/seg_pred'

In [7]:
# assign class labels
class_names =x
class_labels = {class_name:i for i, class_name in enumerate(class_names)}
print(class_labels)

number_classes = len(class_names)

IMAGE_SIZE = (150,150)

{'Common-Rosefinch': 0, 'Asian-Green-Bee-Eater': 1, 'Common-Kingfisher': 2, 'Jungle-Babbler': 3, 'White-Wagtail': 4, 'Indian-Roller': 5, 'Brown-Headed-Barbet': 6, 'Common-Tailorbird': 7, 'Rufous-Treepie': 8, 'White-Breasted-Waterhen': 9, 'Forest-Wagtail': 10, 'Common-Myna': 11, 'Sarus-Crane': 12, 'House-Crow': 13, 'Hoopoe': 14, 'Coppersmith-Barbet': 15, 'Cattle-Egret': 16, 'Indian-Peacock': 17, 'White-Breasted-Kingfisher': 18, 'Gray-Wagtail': 19, 'Ruddy-Shelduck': 20, 'Red-Wattled-Lapwing': 21, 'Indian-Pitta': 22, 'Indian-Grey-Hornbill': 23, 'Northern-Lapwing': 24}


In [6]:
x


['Common-Rosefinch',
 'Asian-Green-Bee-Eater',
 'Common-Kingfisher',
 'Jungle-Babbler',
 'White-Wagtail',
 'Indian-Roller',
 'Brown-Headed-Barbet',
 'Common-Tailorbird',
 'Rufous-Treepie',
 'White-Breasted-Waterhen',
 'Forest-Wagtail',
 'Common-Myna',
 'Sarus-Crane',
 'House-Crow',
 'Hoopoe',
 'Coppersmith-Barbet',
 'Cattle-Egret',
 'Indian-Peacock',
 'White-Breasted-Kingfisher',
 'Gray-Wagtail',
 'Ruddy-Shelduck',
 'Red-Wattled-Lapwing',
 'Indian-Pitta',
 'Indian-Grey-Hornbill',
 'Northern-Lapwing']

In [5]:
x=[]
for folder in os.listdir(train_dir):
    x.append(folder)
#     print(x)

In [8]:
# usecase of os.listdir and os.path.join functions
for folder in os.listdir(train_dir):
    files_path = []
    for file in os.listdir(os.path.join(train_dir,folder)):
        files_path.append(file)
    print(len(files_path))

900
900
900
900
900
900
900
900
900
900
900
900
900
900
900
900
900
900
900
900
900
900
900
900
900


In [9]:
# define load_dataset function to load dataset with labels
def load_dataset():
    # create list of datasets
    datasets = [train_dir, test_dir]
    output = []
    
    for dataset in datasets:
        images1 = []
        labels1 = []
        print(f"loading {dataset}")
        
        for folder in os.listdir(dataset):
            label = class_labels[folder]
            
            for file in tqdm(os.listdir(os.path.join(dataset,folder))):
                image_path = os.path.join(os.path.join(dataset, folder), file)
                
                image_file = cv2.imread(image_path)
                image_file = cv2.cvtColor(image_file, cv2.COLOR_BGR2RGB)
                
                image_file = cv2.resize(image_file, IMAGE_SIZE)
                
                images1.append(image_file)
                labels1.append(label)
                
        images1 = np.array(images1, dtype = 'float32')
        labels1 = np.array(labels1, dtype = 'int32')
        
        output.append((images1, labels1))
        print("Images file have been loaded")
                
    return output 

In [10]:
import os
import cv2
import numpy as np
from tqdm import tqdm

def load_dataset():
    # create list of datasets
    datasets = [train_dir, test_dir]
    output = []
    
    for dataset in datasets:
        images1 = []
        labels1 = []
        print(f"loading {dataset}")
        
        for folder in os.listdir(dataset):
            label = class_labels.get(folder)
            if label is None:
                print(f"Warning: No label found for folder {folder}. Skipping.")
                continue
            
            folder_path = os.path.join(dataset, folder)
            for file in tqdm(os.listdir(folder_path)):
                image_path = os.path.join(folder_path, file)
                
                image_file = cv2.imread(image_path)
                if image_file is None:
                    print(f"Warning: Could not read image {image_path}. Skipping.")
                    continue
                
                image_file = cv2.cvtColor(image_file, cv2.COLOR_BGR2RGB)
                image_file = cv2.resize(image_file, IMAGE_SIZE)
                
                images1.append(image_file)
                labels1.append(label)
                
        images1 = np.array(images1, dtype='float32')
        labels1 = np.array(labels1, dtype='int32')
        
        output.append((images1, labels1))
        print("Images have been loaded.")
                
    return output


In [11]:
# load the data using above functions
((train_images, train_labels), (test_images, test_labels)) = load_dataset()

loading /kaggle/input/seen-dataset-12-class/Seen Datasets/train


 25%|██▌       | 226/900 [00:04<00:12, 56.13it/s]

100%|██████████| 900/900 [00:11<00:00, 76.13it/s]


Images have been loaded.
loading /kaggle/input/seen-dataset-12-class/Seen Datasets/val


100%|██████████| 300/300 [00:04<00:00, 73.25it/s]


Images have been loaded.


In [12]:
train_images.shape

(22499, 150, 150, 3)

In [14]:
print("train dataset size")
len(train_images), len(train_labels)

train dataset size


(22499, 22499)

In [13]:
print("test dataset size")
len(test_images), len(test_labels)

test dataset size


(7500, 7500)

# Pre-processing

In [20]:
# shape of image dataset is in 4D 
train_images.shape, test_images.shape

((17000, 150, 150, 3), (5000, 150, 150, 3))

In [16]:
# shuffle train and test datasets
(train_images, train_labels) = shuffle(train_images, train_labels, random_state=45)

In [17]:
(test_images, test_labels) = shuffle(test_images, test_labels, random_state=45)

In [18]:
# keeping 6000 data instances for training and testing our model
train_images = train_images[:17000]
train_labels = train_labels[:17000]

In [19]:
test_images = test_images[:5000]
test_labels = test_labels[:5000]

In [21]:
# normalize the images dataset
trn_images_norm = train_images/255.0
tst_images_norm = test_images/255.0

In [70]:
model = models.efficientnet_b3(pretrained=False)

# Modify the classifier to have 25 output classes instead of 1000
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 25)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [71]:
def initialize_weights(m):
    if isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
        if m.bias is not None:
            nn.init.constant_(m.bias, 0)
    elif isinstance(m, nn.BatchNorm2d):
        nn.init.constant_(m.weight, 1)
        nn.init.constant_(m.bias, 0)
    elif isinstance(m, nn.Linear):
        nn.init.normal_(m.weight, 0, 0.01)
        nn.init.constant_(m.bias, 0)

# Apply the weight initialization
model.apply(initialize_weights)

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 40, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(40, 40, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=40, bias=False)
            (1): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(40, 10, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(10, 40, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActiv

In [75]:
# Input: 224x224 RGB image
# |
# |--> [Conv7x7, 64, stride 2] -> [MaxPool, 3x3, stride 2]
# |
# |--> Stage 1: [2 x {3x3, 64} residual blocks]
# |
# |--> Stage 2: [2 x {3x3, 128} residual blocks, first with stride 2]
# |
# |--> Stage 3: [2 x {3x3, 256} residual blocks, first with stride 2]
# |
# |--> Stage 4: [2 x {3x3, 512} residual blocks, first with stride 2]
# |
# |--> Global Average Pooling
# |
# |--> Fully Connected (1000)
# |
# Output: 1000 class scores


In [72]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 40, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(40, 40, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=40, bias=False)
            (1): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(40, 10, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(10, 40, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActiv

In [73]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    with tqdm(total=len(trainloader), desc=f'Epoch {epoch+1}/{num_epochs}', unit='batch') as pbar:
        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)
            labels = labels.long()  # Convert labels to LongTensor

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            pbar.set_postfix(loss=running_loss/(pbar.n+1))
            pbar.update(1)
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(trainloader)}")

print("Training complete.")

Epoch 1/10: 100%|██████████| 266/266 [01:25<00:00,  3.12batch/s, loss=2.91]


Epoch 1/10, Loss: 2.91277663779438


Epoch 2/10: 100%|██████████| 266/266 [01:27<00:00,  3.03batch/s, loss=2.57]


Epoch 2/10, Loss: 2.5701177783478473


Epoch 3/10: 100%|██████████| 266/266 [01:29<00:00,  2.97batch/s, loss=2.28]


...

Epoch 4/10, Loss: 2.0131376175055826


Epoch 5/10: 100%|██████████| 266/266 [01:30<00:00,  2.95batch/s, loss=1.78]


Epoch 5/10, Loss: 1.7832959666287989


Epoch 6/10:  44%|████▍     | 118/266 [00:40<00:50,  2.91batch/s, loss=1.62]


KeyboardInterrupt: 

In [1]:
model=0

In [86]:
device=torch.device('cuda')

In [74]:
import torch
import torch.nn as nn


# ConvBlock
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, groups=1, act=True, bias=False):
        super().__init__()
        """ If k = 1 -> p = 0, k = 3 -> p = 1, k = 5, p = 2. """
        padding = kernel_size // 2
        self.c = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride, padding=padding, bias=bias, groups=groups)
        self.bn = nn.BatchNorm2d(out_channels)
        self.silu = nn.SiLU() if act else nn.Identity()

    def forward(self, x):
        x = self.silu(self.bn(self.c(x)))
        return x

# Squeeze-and-Excitation Block
class SeBlock(nn.Module):
    def __init__(self, in_channels, r):
        super().__init__()
        C = in_channels
        self.globpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc1 = nn.Linear(C, C//r, bias=False)
        self.fc2 = nn.Linear(C//r, C, bias=False)
        self.silu = nn.SiLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        """ x shape: [N, C, H, W]. """ 
        f = self.globpool(x)
        f = torch.flatten(f,1)
        f = self.silu(self.fc1(f))
        f = self.sigmoid(self.fc2(f))
        f = f[:,:,None,None]
        """ f shape: [N, C, 1, 1] """ 

        scale = x * f
        return scale

# MBConv
class MBConv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, exp, r):
        super().__init__()
        exp_channels = in_channels * exp
        self.add = in_channels == out_channels and stride == 1
        self.c1 = ConvBlock(in_channels, exp_channels, 1, 1) if exp > 1 else nn.Identity()
        self.c2 = ConvBlock(exp_channels, exp_channels, kernel_size, stride, exp_channels)
        self.se = SeBlock(exp_channels, r)
        self.c3 = ConvBlock(exp_channels, out_channels, 1, 1, act=False)

        " Stochastic Depth module with default survival probability 0.5. "
        self.sd = StochasticDepth()

    def forward(self, x):
        f = self.c1(x)
        f = self.c2(f)
        f = self.se(f)
        f = self.c3(f)

        if self.add:
            f = x + f

        f = self.sd(f)

        return f

# Classfier
class Classifier(nn.Module):
    """ Last stage with Average Pooling and Fully-Connected layer. """ 
    def __init__(self, in_channels, classes, p):
        super().__init__()
        self.pool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(in_channels, classes)
        self.dropout = nn.Dropout(p)

    def forward(self, x):
        x = self.dropout(self.pool(x))
        x = torch.flatten(x, 1)
        return self.fc(x)

# EfficientNet
class EfficientNet(nn.Module):
    def __init__(
        self, 
        model_config : str,
        in_channels : int = 3, 
        classes : int = 25,
        show : str = False
        ):
        super().__init__()
        self.show = show # if True print the output dim between layers 
        config = Config()
        stages = config.stages
        phis = config.phis[model_config]

        """ Parameters. """
        phi, res, p = phis
        self._calculate_coef(phi)

        """ Network. """
        self.net = nn.ModuleList([])
        self.channels = []

        """ First stage Conv3x3. """
        f, c, l, k, s, exp = stages[0]
        self._add_layer(3, f, c, l, k, s)

        """ 2-8 stages with MBConvs. """
        for i in range(1, len(stages)-1):
            if i == 1:
                r = 4
            else:
                r = 24

            f, c, l, k, s, exp = stages[i]
            self._add_layer(self.channels[-1], f, c, l, k, s, exp, r)

        """ Last stage Conv1x1 + Classifier. """
        f, c, l, k, s, exp = stages[-1]
        self._add_layer(self.channels[-1], f, c, l, k, s)
        self.net.append(Classifier(self.channels[-1], classes, p))

    def forward(self, x):
        ''' Fancy way to print shapes of certain stages. '''
        i = 1
        for F in self.net:
            in_feat, h, w = x.shape[1:]
            
            
            x = F(x)
            if in_feat != x.shape[1] and i < 10:
                if self.show : print("Stage {} -> ".format(i), [x.shape[1], h, w])
                i += 1
        return x

    def _add_layer(self, in_channels, f, c, l, k, s, *args):
        c, l = self._update_feat(c, l)
        if l == 1:
            self.net.append(f(in_channels, c, k, s, *args))
        else:
            """ First layer with stride 1. """
            self.net.append(f(in_channels, c, k, 1, *args))
            
            """ Another layers with stride 1. """
            for _ in range(l-2):
                self.net.append(f(c, c, k, 1, *args))                
        
            """ Final layer with stride s(1 or 2). """
            self.net.append(f(c, c, k, s, *args))

        self.channels.append(c)
                
    def _calculate_coef(self, phi, alpha=1.2, beta=1.1):
        self.d = alpha**phi
        self.w = beta**phi

    def _update_feat(self, c, l):
        return int(c * self.w), int(l * self.d)

class Config:
    stages = [
            # [Operator(f), Channels(c), Layers(l), Kernel(k), Stride(s), Expansion(exp)]
            [ConvBlock, 32, 1, 3, 2, 1], 
            [MBConv, 16, 1, 3, 1, 1],
            [MBConv, 24, 2, 3, 2, 6],
            [MBConv, 40, 2, 5, 2, 6],
            [MBConv, 80, 3, 3, 2, 6],
            [MBConv, 112, 3, 5, 1, 6],
            [MBConv, 192, 4, 5, 2, 6],
            [MBConv, 320, 1, 3, 1, 6],
            [ConvBlock, 1280, 1, 1, 1, 0]
    ]

    phis = {
            # BX : (phi, resolution, dropout) 
            "B0" : (0, 224, 0.2), 
            "B1" : (0.5, 240, 0.2),
            "B2" : (1, 260, 0.3),
            "B3" : (2, 300, 0.3),
            "B4" : (3, 380, 0.4),
            "B5" : (4, 456, 0.4),
            "B6" : (5, 528, 0.5),
            "B7" : (6, 600, 0.5)
    }



In [75]:
import torch
from torch import Tensor
import torch.nn as nn 

class StochasticDepth(nn.Module):
    " Stochastic Depth / Drop Path module which takes survival probability(p) and returns input with *dropped* examples in the batch. "
    
    def __init__(
        self,
        p : float = 0.5
        ):
        super().__init__()
        self.p = p
        
    def forward(self, x: Tensor) -> Tensor:
        mask_shape = (x.shape[0],) + (1,)*(x.ndim - 1)
        # mask shape: [batch, 1, 1, 1]
        mask = torch.empty(mask_shape, device=x.device).bernoulli_(self.p) / self.p
        
        if self.training : x = mask * x
        return x 



In [76]:
# model=0

In [79]:

model_config = "B3" 
model = EfficientNet(model_config)

device = torch.device('cuda')
model.to(device)

EfficientNet(
  (net): ModuleList(
    (0): ConvBlock(
      (c): Conv2d(3, 38, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(38, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (silu): SiLU()
    )
    (1): MBConv(
      (c1): Identity()
      (c2): ConvBlock(
        (c): Conv2d(38, 38, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=38, bias=False)
        (bn): BatchNorm2d(38, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (silu): SiLU()
      )
      (se): SeBlock(
        (globpool): AdaptiveAvgPool2d(output_size=(1, 1))
        (fc1): Linear(in_features=38, out_features=9, bias=False)
        (fc2): Linear(in_features=9, out_features=38, bias=False)
        (silu): SiLU()
        (sigmoid): Sigmoid()
      )
      (c3): ConvBlock(
        (c): Conv2d(38, 19, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(19, eps=1e-05, momentum=0.1, affine=True, track_runni

In [80]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

EfficientNet(
  (net): ModuleList(
    (0): ConvBlock(
      (c): Conv2d(3, 38, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(38, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (silu): SiLU()
    )
    (1): MBConv(
      (c1): Identity()
      (c2): ConvBlock(
        (c): Conv2d(38, 38, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=38, bias=False)
        (bn): BatchNorm2d(38, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (silu): SiLU()
      )
      (se): SeBlock(
        (globpool): AdaptiveAvgPool2d(output_size=(1, 1))
        (fc1): Linear(in_features=38, out_features=9, bias=False)
        (fc2): Linear(in_features=9, out_features=38, bias=False)
        (silu): SiLU()
        (sigmoid): Sigmoid()
      )
      (c3): ConvBlock(
        (c): Conv2d(38, 19, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(19, eps=1e-05, momentum=0.1, affine=True, track_runni

In [81]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    with tqdm(total=len(trainloader), desc=f'Epoch {epoch+1}/{num_epochs}', unit='batch') as pbar:
        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)
            labels = labels.long()  # Convert labels to LongTensor

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            pbar.set_postfix(loss=running_loss/(pbar.n+1))
            pbar.update(1)
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(trainloader)}")

print("Training complete.")

Epoch 1/10: 100%|██████████| 266/266 [01:44<00:00,  2.55batch/s, loss=3.29]


Epoch 1/10, Loss: 3.292560879449199


Epoch 2/10: 100%|██████████| 266/266 [01:46<00:00,  2.49batch/s, loss=3.23]


Epoch 2/10, Loss: 3.231930148332639


Epoch 3/10: 100%|██████████| 266/266 [01:47<00:00,  2.48batch/s, loss=3.23]


...

Epoch 5/10, Loss: 3.2265133803948425


Epoch 6/10: 100%|██████████| 266/266 [01:46<00:00,  2.49batch/s, loss=3.23]


Epoch 6/10, Loss: 3.2275688791633548


Epoch 7/10:  16%|█▌        | 43/266 [00:17<01:32,  2.41batch/s, loss=3.23]


KeyboardInterrupt: 

In [100]:
class AlexNet(nn.Module):
    def __init__(self, num_classes=25):
        super().__init__()
        self.features_extraction = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),  
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=3, stride=2),
              
            
            nn.Conv2d(64, 192, kernel_size=5, padding=2),  
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(192),
            nn.MaxPool2d(kernel_size=3, stride=2),
            
            
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(384),
            
            nn.Conv2d(384, 256, kernel_size=3, padding=1),  
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(256),
            
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.AdaptiveAvgPool2d((6, 6)),  
        )
        self.classifier = nn.Sequential(
            nn.Dropout(0.1),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(4096),
            
            nn.Dropout(0.1),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(4096),
            
            nn.Dropout(0.1),
            nn.Linear(4096, 512),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(512),
            
            nn.Linear(512, num_classes),
        )

    def forward(self, x):
        x = self.features_extraction(x)
        x = x.view(x.size(0), -1)
        return self.classifier(x)
    

model=AlexNet()
# model = nn.DataParallel(model, device_ids = [0,1]) # to use multipleGPU
model = model.to(device)

In [70]:
import torch


model_config = "B0" # ["B0", "B1", "B2", "B3", "B4", "B5", "B6", "B7"] -> are available
model = EfficientNet(model_config)



tensor([[ 0.0144, -0.0247,  0.0072, -0.0172, -0.0075,  0.0021,  0.0141,  0.0129,
         -0.0070, -0.0119, -0.0115,  0.0161, -0.0154, -0.0110,  0.0213,  0.0036,
         -0.0031, -0.0131, -0.0105, -0.0057, -0.0256, -0.0274,  0.0150, -0.0059,
          0.0128]], grad_fn=<AddmmBackward0>)
